In [4]:
# 约瑟夫环
# 创建两个类，面向人和环
# 指定起始位置，间隔step的人出列

import csv
import zipfile
import copy

class Person(object):
    def __init__(self, name, age):
        self.name = name 
        self.age = age 

class Ring:
    def __init__(self, start, step):
        self.start = start
        self.step = step
        self.people = []
    
     # 增功能
    def append(self,obj):
        if(len(self.people) > Ring.MAX_size):
            raise Exception("exceed the max cap of people.")
        self.people.append(obj)
   
    # 删功能
    def pop(self):
        self.people.pop(obj)
    
    # 查找
    def query_list(self):
        return self.people
    
    def reset(self):  
        self.current_id = self.start - 1
        self.temp = copy.copy(self.people)
        self.length = len(self.people)

    def next(self):
        size = len(self.temp)
        if size == 0:
            return None
            
        self.current_id = (self.current_id + self.step - 1) % size
        index = self.temp.pop(self.current_id)
        
        return index

    def iter(self):
        # self.temp = copy.copy(self.people)
        size = len(self.temp)
        if size == 0:
            return None

        for i in range(size):
            self.current_id = (self.current_id + self.step - 1) % size
            index = self.temp.pop(self.current_id)
            yield index

    @classmethod
    def create_from_txt(cls,path):

        data = [] 
        with open(path,'r') as txt_file:
            for line in txt_file:
                temp = line.strip()
                name = (temp[ : temp.index(',')])
                age = int(temp[temp.index(',') + 1 : ])
                data.append(Person(name, age))
        return data
    
    @staticmethod
    def create_from_csv(cls,path): 

        data = []
        with open(path,'r') as csv_file:
            csv_reader = csv.reader(csv_file)
            for line in csv_reader:
                name = line[0]
                age = line[1]
                data.append(Person(name, age))
        return data

class Read:
    # 打开文件
    def open(self,filename):
        self.filename = filename
        self.f = open(self.filename, 'r')
    
    # 关闭文件
    def close(self):
        self.f.close()

    # 读取txt文件
    def read_txt(self):
        if '.txt' in self.filename:
            for line in self.f:
                temp = line.strip()
                name = (temp[ : temp.index(',')])
                age = int(temp[temp.index(',') + 1 : ])
                return name, age

    # 读取csv文件 
    def read_csv(self):
        if '.csv' in self.filename:
            csv_reader = csv.reader(self.f)
            for line in csv_reader:
                name, age = line
                return name, age

    # 读取zip文件
    def read_zipfile():

        z = zipfile.ZipFile("zipfile.zip", "r")
        #打印zip文件中的文件列表
        for filename in z.namelist( ):
            print(filename)
        #读取zip文件中的第一个文件
        first_file_name = z.namelist()[0]
        content = z.read(first_file_name)
        print(first_file_name)
        print(content)

In [3]:
ring = Ring(2,3)
# 通过txt文件写入
f = Read()
f.open('people.txt')
data = f.read_txt()
for line in data:
    name, age = line
    ring.append(Person(name, age))
f.close()

# ring.append(Person("John", 18))
# ring.append(Person("Bob", 19))
# ring.append(Person("Jerry", 20))
# ring.append(Person("Mark", 21))
# ring.append(Person("Jack", 22))
# ring.append(Person("Neil", 23))
# ring.append(Person("Oscar", 24))


ring.reset()
for i in range(ring.length):
    index = ring.next()
    if index:
        print("第{}个出列的人：{}；年龄：{}". format(i+1, index.name, index.age))

print('-'*25)

ring.reset()
generator = ring.iter()
for i in range(ring.length):
    index = next(generator)
    if index:
        print("第{}个出列的人：{}；年龄：{}". format(i+1, index.name, index.age))

第1个出列的人："Mark"；年龄：21
第2个出列的人："Oscar"；年龄：24
第3个出列的人："Jerry"；年龄：20
第4个出列的人："John"；年龄：18
第5个出列的人："Neil"；年龄：23
第6个出列的人："Bob"；年龄：19
第7个出列的人："Jack"；年龄：22
-------------------------
第1个出列的人："Mark"；年龄：21
第2个出列的人："Oscar"；年龄：24
第3个出列的人："Jerry"；年龄：20
第4个出列的人："John"；年龄：18
第5个出列的人："Neil"；年龄：23
第6个出列的人："Bob"；年龄：19
第7个出列的人："Jack"；年龄：22
